In [ ]:
import sys
sys.path.insert(0, "..")

import scikitplot as skplt 
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IA.utils import mapping, parameter_range
from scipy import stats
import math
import swifter
import matplotlib.pyplot as plt
plt.style.reload_library()
plt.style.use(['science'])

In [ ]:
!ls -lah analysis/not_uploaded/IA/ | grep class

In [ ]:
df = pd.read_csv("analysis/not_uploaded/IA/.scratch.ckpts.IA.pexels.scores-one.change_class.epoch-5.pth.txt")

for col in df.columns:
    try:
        df[col] = df[col].swifter.apply(lambda row: eval(row)[0])
    except:
        pass

print(df.columns)
df

In [ ]:
df["intensity"] = [float("nan")]*len(df)
df.loc[df["level"] == 0, "intensity"] = 0
for t in ["styles", "technical", "composition"]:
    for dist in mapping[t].keys():
        for pol in mapping[t][dist].keys():
            if pol == "neg":
                for lvl, change in enumerate(reversed(mapping[t][dist][pol])):
                    df.loc[(df["distortion"] == change.split(";")[0])&(df["level"] == float(change.split(";")[1])), "intensity"] = -(lvl+1)
            if pol == "pos":
                for lvl, change in enumerate(mapping[t][dist][pol]):
                    df.loc[(df["distortion"] == change.split(";")[0])&(df["level"] == float(change.split(";")[1])), "intensity"] = lvl+1
df["intensity"] = df["intensity"].astype(int)
df

In [ ]:
df = df[(df["intensity"] == 5) | (df["intensity"] == -5)]
df

In [ ]:
dist_type = "composition"

df = df[df["distortion"].isin(mapping[dist_type].keys())]

In [ ]:
def find_class(row:pd.Series, dist_type:str):
    relevant_cols = [c for c, _ in row.iteritems() if dist_type in c and "class" in c]
    m = -1
    m_c = ""
    for c in relevant_cols:
        if row[c] > m:
            m = row[c]
            m_c = c
    return m_c.split("_")[1]

In [ ]:
df

In [ ]:
df[f"{dist_type}_class"] = df.apply(lambda row: find_class(row, dist_type), axis=1)
df

In [ ]:
df = df[["distortion", f"{dist_type}_class", "intensity"]]
df

In [ ]:
if dist_type == "technical":
    df["distortion"] = df["distortion"].swifter.apply(lambda row: row.split("_")[0])

In [ ]:
accuracy_score(df["distortion"], df[f"{dist_type}_class"])